In [1]:
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from qcodes.dataset.plotting import plot_dataset,plot_by_id
from qcodes.logger.logger import start_all_logging
# from qcodes.plots.pyqtgraph import QtPlot
from qcodes.loops import Loop
from time import sleep
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import qcodes as qc
import numpy as np
import datetime
import time
import json
import sys
import os

# Agilent

In [2]:
sys.path.append('C:\QCodes_Driver_And_Scripts\Drivers')

from qcodes.instrument_drivers.yokogawa.GS200 import GS200

from Agilent_E8257D import E8257D
from RS_SGS_100A import SGS100A
from RS_SMB_100A import SMB100A
from HP3245A import HP3245A
from RS_FSV import RS_FSV
from VNA_RS import VNA


# try:
#     VNA = VNA('ZNB20', 'TCPIP0::192.168.10.2::inst0::INSTR',
#                init_s_params=False,terminator='\n')

# except KeyError as er:


#     VNA.close_all() #Disconnect and irreversibly tear down the instrument
#     VNA = VNA('ZNB20', 'TCPIP0::192.168.10.2::inst0::INSTR',
#                init_s_params=False,terminator='\n')

# try:
#     pump_source=SGS100A('pump_source','TCPIP0::192.168.10.56::inst0::INSTR')

# except KeyError as er:
#     SGS100A.close_all()
#     pump_source=SGS100A('pump_source','TCPIP0::192.168.10.65::inst0::INSTR')
       
try:

    twpa_source=SGS100A('twpa_pump','TCPIP0::192.168.10.56::inst0::INSTR')

except KeyError as er:

    SMB100A.close_all()
    twpa_source=SGS100A('twpa_pump','TCPIP0::192.168.10.56::inst0::INSTR')
     
    
try:
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")


except KeyError as er:

    GS200.close_all() #Disconnect and irreversibly tear down the instrument
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")

try:
    MW_source=E8257D('mw_source','TCPIP0::192.168.10.70::inst0::INSTR')

except KeyError as er:

    E8257D.close_all()
    MW_source=E8257D('mw_source','TCPIP0::192.168.10.70::inst0::INSTR')
try:
    FSV = RS_FSV('rs_fsv', 'TCPIP0::192.168.10.66::inst0::INSTR')

except KeyError as er:


    RS_FSV.close_all() #Disconnect and irreversibly tear down the instrument
    FSV =  RS_FSV('rs_fsv', 'TCPIP0::192.168.10.66::inst0::INSTR')


Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/114045, firmware:4.2.76.0-4.30.046.295) in 0.06s
Connected to: YOKOGAWA GS210 (serial:90Y438440, firmware:2.02) in 0.02s
Connected to: Agilent Technologies E8257D (serial:US45470698, firmware:C.06.28) in 0.03s
Connected to: Rohde&Schwarz FSV3030 (serial:1330.5000K30/101857, firmware:1.80) in 0.01s


In [3]:
def measure(avg,pow_vec, exp, station):

    meas = qc.Measurement(exp=exp, station=station)
    meas.register_parameter(MW_source.power)
    meas.register_custom_parameter(name='power_on',setpoints=[MW_source.power])
    meas.register_custom_parameter(name='power_off',setpoints=[MW_source.power])
    
    with meas.run() as datasaver:
        
        run_id=datasaver.dataset.run_id
        qc.load_by_run_spec(captured_run_id=run_id).add_metadata('parameter_snap',json.dumps(parameter_snap))
        
        for poww in tqdm(pow_vec):
            
            MW_source.power(poww)
            
            get_p = float(MW_source.power.get())
            
            get_s_off  = np.zeros(FSV.n_points())
            get_s      = np.zeros(FSV.n_points())
            
            for n in tqdm(range(avg),leave=False):
                
                MW_source.status('OFF')
                
                get_s_off =  get_s_off + 1.e-3*10**(FSV.spectrum.get()/10)
                
                MW_source.status(src_status.upper())

                get_c = FSV.freq_axis.get()

                get_s = get_s +  1.e-3*10**(FSV.spectrum.get()/10)
                
                sleep(0.01)

            datasaver.add_result((MW_source.power, get_p),("power_on",np.average(get_s/avg)),("power_off",np.average(get_s_off/avg)))

          


In [4]:
start_all_logging()

# Create a station
station = qc.Station()
station.add_component(FSV)
station.add_component(twpa_source)
station.add_component(I_source)
# station.add_component(I_twpa)
station.add_component(MW_source)

station.snapshot()
station.components

# Experiment details
user='DF'
date=datetime.datetime.today().strftime('%Y_%m_%d')
description='FSV_zero_span_vs_agilent_pow'
database_name = date+"_"+user+"_"+description

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nicolas.ROCH\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nicolas.ROCH\.qcodes\logs\221209-24876-qcodes.log


2022-12-09 14:57:13,876 ¦ qcodes.instrument.instrument_base ¦ WARNING ¦ instrument_base ¦ snapshot_base ¦ 292 ¦ [gs200_program(GS200Program)] Snapshot: Could not update parameter: trigger


In [5]:
script_dir=os.path.dirname(os.path.abspath("__file__"))
data_dir=os.path.join('C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\'+date)

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

data_dir=data_dir +'\\'+description

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

In [6]:
qc.initialise_or_create_database_at(data_dir+'\\'+database_name)
qc.config.core.db_location

'C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\2022_12_09\\FSV_zero_span_vs_agilent_pow\\2022_12_09_DF_FSV_zero_span_vs_agilent_pow'

In [7]:
current =  -2.761#mA
# current = 0  #mA

# TWPA PUMP PARAMETERS
pump_freq    = 4.998                #GHz
pump_power   = 0.5                #dBm
pump_status  = 'on'
twpa_current = -0.5               #mA

print('\nMeasurement with TWPA {}'.format(pump_status))


# MW SOURCE
src_center     = 9.155
src_span       = .4
src_points     = 100
src_points     = 1
src_frequency  = np.linspace(src_center-0.5*src_span,src_center + 0.5*src_span, src_points) #GHz
src_frequency  = np.array([src_center])
src_power      = np.linspace(-20.,0.,50)
src_status     = 'on'

# Spectral Analyser parameters
FSV_rbw         =  10*1.e6 #Hz
FSV_ref         = -10      #dB
FSV_avg         =  0
FSV_att         =  0       #dB
FSV_center      =  3.1   #GHz
FSV_span        =  0       #0 span mode
FSV_npt         =  250
sweep_time_min  = FSV_npt / FSV_rbw
trace_average   =  32*2
# trace_average   =  10

FSV_time_sweep  = 1.
FSV_time_sweep  = max(sweep_time_min,FSV_time_sweep)

if FSV_time_sweep < sweep_time_min :
    raise ValueError("sweeptime to small with respect to the sweep points and rbw")
print('\ntime estimation for all measurement : {} '.format(datetime.timedelta(seconds=2*1.25 *len(src_power)*FSV_time_sweep*trace_average)))


Measurement with TWPA on

time estimation for all measurement : 2:13:20 


In [8]:
if I_source.output()=='on':
    I_source.output('off')

I_source.source_mode('CURR')
I_source.range(100e-3)
I_source.current(current*1.e-3)
I_source.output('on')

FSV.res_BW(FSV_rbw)
FSV.center_freq(FSV_center*1.e9)
FSV.averages(FSV_avg)
FSV.n_points(FSV_npt)
FSV.span(FSV_span*1.e9)
FSV.ref_level(FSV_ref)
FSV.input_att(FSV_att)
FSV.write('SWE:TYPE AUTO')
FSV.sweep_time(FSV_time_sweep)
FSV.detector('rms')

MW_source.frequency(src_frequency[0]*1.e9)
MW_source.power(src_power[0])
MW_source.status(src_status.upper())
MW_source.set_gui_update('on')

twpa_source.frequency(pump_freq*1.e9)
twpa_source.power(pump_power)
twpa_source.status(pump_status.lower())

parameter_snap={}
# Snapshot parameters
parameter_snap['twpa_pump'] = {'freq':pump_freq,'pump_power':pump_power,'pump_status' : pump_status,'current' : twpa_current}
parameter_snap['MW_source'] = {'freq':src_frequency[0]}
parameter_snap['FSV']={'center_freq':FSV_center,'average':FSV_avg,'res_bandwidth':FSV_rbw,'Npoints':int(FSV.n_points()),'ref':FSV_ref,'att':FSV_att,'span':FSV_span,'sweeptime':FSV_time_sweep,'trace_average':trace_average}
parameter_snap['flux']={'current': current}

# Load experiment
exp_name = 'FSV_zero_span_vs_agilent_pow'
sample_name = 'B5_G3_GALV_SQ200_1ST'
exp=qc.load_or_create_experiment(experiment_name=exp_name,sample_name=sample_name)

measure(avg=trace_average,pow_vec=src_power,exp=exp, station=station)

twpa_source.status('off')
MW_source.status('OFF')
I_source.output('off')

Starting experimental run with id: 4. 


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]